# 股票预测模型工作流

---
### 工作流说明
1.  **阶段零 (Setup)**: 导入库、加载配置。
2.  **阶段一 (Data Pipeline)**: 独立运行。负责处理并保存数据，生成 L2 特征数据缓存。
3.  **阶段二 (Model Pipeline)**: 独立运行。包含三个子步骤：
    - **2.1 HPO**: 自动调参。
    - **2.2 (预处理)**: 智能地加载或生成 L3 预处理数据缓存
    - **2.3 (模型训练)**: 使用 L3 缓存进行高效的模型训练。
    - **2.4 (评估)**: 对训练结果进行聚合与可视化。

## 0. 通用设置与导入

In [1]:
import sys
import json
import yaml
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import torch
import joblib
from tqdm.autonotebook import tqdm
from sklearn.preprocessing import StandardScaler
from utils.config_utils import load_and_merge_configs_for_notebook

# --- 1. 环境与路径设置 ---
project_root = str(Path().resolve())
if project_root not in sys.path:
    print(f"将项目根目录添加到 sys.path: {project_root}")
    sys.path.append(project_root)

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# --- 2.  加载配置和所有模块 ---
# a. 加载配置
config = load_and_merge_configs_for_notebook()

# b. 手动加载所有模块
print("--- 正在加载所有项目模块... ---")
try:
    from main_train import (
        run_all_data_pipeline, run_preprocess_l3_cache, run_hpo_train,
        run_all_models_train, run_performance_evaluation, run_results_visualization,
    )
    from data_process.get_data import initialize_apis, shutdown_apis, get_full_feature_df, get_latest_global_data
    from data_process.save_data import run_data_pipeline, get_processed_data_path
    from model.build_models import run_training_for_ticker
    from utils.hpo_utils import run_hpo_for_ticker
    from model.builders.model_fuser import ModelFuser
    from model.builders.lgbm_builder import LGBMBuilder
    from model.builders.lstm_builder import LSTMBuilder, LSTMModel
    from model.builders.tabtransformer_builder import TabTransformerBuilder, TabTransformerModel
    from risk_management.risk_manager import RiskManager
    from utils.date_utils import resolve_data_pipeline_dates
    from utils.encoding_utils import encode_categorical_features
    from utils.file_utils import find_latest_artifact_paths
    from utils.ml_utils import walk_forward_split
    print("INFO: 项目模块导入成功。")
except ImportError as e:
    raise ImportError(f"模块导入失败，请确保 Notebook 的运行目录在项目根目录。错误: {e}")

# c. 构建 modules 字典
modules = {
    'initialize_apis': initialize_apis, 'shutdown_apis': shutdown_apis,
    'get_full_feature_df': get_full_feature_df, 'get_latest_global_data': get_latest_global_data,
    'run_data_pipeline': run_data_pipeline, 'get_processed_data_path': get_processed_data_path,
    'run_training_for_ticker': run_training_for_ticker, 'run_hpo_for_ticker': run_hpo_for_ticker,
    'ModelFuser': ModelFuser, 'LGBMBuilder': LGBMBuilder, 'LSTMBuilder': LSTMBuilder, 'LSTMModel': LSTMModel,
    'TabTransformerBuilder': TabTransformerBuilder, 'TabTransformerModel': TabTransformerModel,
    'RiskManager': RiskManager,
    'resolve_data_pipeline_dates': resolve_data_pipeline_dates,
    'encode_categorical_features': encode_categorical_features,
    'find_latest_artifact_paths': find_latest_artifact_paths,
    'walk_forward_split': walk_forward_split,
    'pd': pd, 'torch': torch, 'joblib': joblib, 'tqdm': tqdm, 'StandardScaler': StandardScaler, 'Path': Path, 'yaml': yaml, 'json': json
}


C:\Users\Akari\AppData\Local\Temp\ipykernel_28204\2077795851.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


将项目根目录添加到 sys.path: D:\Project\Command\Python\Neural\Wolf_of_Wall_Street
--- 开始为 Notebook 加载和合并所有配置文件 ---
  - [DEBUG] 主配置 'config.yaml' 已加载。

  - [DEBUG] 正在合并 'data/default'...

  - [DEBUG] 正在合并 'features/default'...

  - [DEBUG] 正在合并 'labeling/raw_return'...

  - [DEBUG] 正在合并 'model/lgbm'...

  - [DEBUG] 正在合并 'hpo/default'...

  - [DEBUG] 正在合并 'backtest/default'...

  - [DEBUG] 正在合并 'application/default'...

  - [DEBUG] 正在合并 '_self_'...

--- 所有配置文件合并完成 ---
--- 正在加载所有项目模块... ---
INFO: 项目模块导入成功。


# **阶段一：数据准备与特征工程**

In [2]:
USE_LATEST_DATE = True
if config and modules:
    print(f"--- INFO: 日期模式 -> {'使用最新日期' if USE_LATEST_DATE else '使用配置文件中的固定日期'} ---")
    run_all_data_pipeline(config, modules, use_today_as_end_date=USE_LATEST_DATE)

--- INFO: 日期模式 -> 使用最新日期 ---
=== 阶段一：数据准备与特征工程 ===
INFO: 已启用动态日期模式，将使用今天的日期 '2025-11-01' 作为 end_date。
INFO: 日期已解析. Start Date: 2010-11-01, End Date: 2025-11-01
INFO: 正在尝试登录 Baostock...
login success!
INFO: Baostock API 登录成功。
INFO: 未在配置中提供有效的 Tushare Token，将跳过 Tushare 相关数据。
开始执行数据管道协调任务...

需要为以下 7 只股票生成新数据: ['TCL科技 (000100.SZ)', '兴业矿业 (000426.SZ)', '孚日股份 (002083.SZ)', '宜华健康 (000150.SZ)', '新宁物流 (300013.SZ)', '佳云科技 (300242.SZ)', '精功科技 (002006.SZ)']

--- 正在准备所有全局市场数据 (此过程只运行一次) ---
--- 开始生成市场广度数据 ---
  - 正在获取指数成分股列表...


下载成分股日线数据:   0%|          | 0/300 [00:00<?, ?it/s]

  - 正在从 Baostock 下载 sh.600000 (sh.600000) 的日线行情...
  - INFO: 已将 sh.600000 (sh.600000) 的数据缓存至 data\data_cache\raw_ohlcv\raw_sh_600000_2010-11-01_2025-09-30.pkl
  - 正在从 Baostock 下载 sh.600009 (sh.600009) 的日线行情...
  - INFO: 已将 sh.600009 (sh.600009) 的数据缓存至 data\data_cache\raw_ohlcv\raw_sh_600009_2010-11-01_2025-09-30.pkl
  - 正在从 Baostock 下载 sh.600010 (sh.600010) 的日线行情...
  - INFO: 已将 sh.600010 (sh.600010) 的数据缓存至 data\data_cache\raw_ohlcv\raw_sh_600010_2010-11-01_2025-09-30.pkl
  - 正在从 Baostock 下载 sh.600011 (sh.600011) 的日线行情...
  - INFO: 已将 sh.600011 (sh.600011) 的数据缓存至 data\data_cache\raw_ohlcv\raw_sh_600011_2010-11-01_2025-09-30.pkl
  - 正在从 Baostock 下载 sh.600015 (sh.600015) 的日线行情...
  - INFO: 已将 sh.600015 (sh.600015) 的数据缓存至 data\data_cache\raw_ohlcv\raw_sh_600015_2010-11-01_2025-09-30.pkl
  - 正在从 Baostock 下载 sh.600016 (sh.600016) 的日线行情...
  - INFO: 已将 sh.600016 (sh.600016) 的数据缓存至 data\data_cache\raw_ohlcv\raw_sh_600016_2010-11-01_2025-09-30.pkl
  - 正在从 Baostock 下载 sh.600018 (sh.600018) 的日线行

d:\Project\Command\Python\Neural\Wolf_of_Wall_Street\data_process\get_data.py:107: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns_df = closes_df.pct_change().fillna(0)


--- 市场广度数据已生成并缓存至 data\data_cache\market_breadth_2010-11-01_2025-09-30.pkl ---
INFO: 正在下载 4 个外部市场的数据: ['SPY', 'QQQ', 'GLD', 'TLT']
  - 正在为 SPY 获取美股数据...
  - 正在从 Yahoo Finance 下载 SPY 的数据...


...

  - 警告 [YF]: 下载 TLT 数据失败 (尝试 1/3). 将在 0.50 秒后重试. 错误: yfinance returned an empty DataFrame for TLT.


d:\Project\Command\Python\Neural\Wolf_of_Wall_Street\data_process\get_data.py:308: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start_date, end=end_date_inclusive, progress=False)

1 Failed download:
['TLT']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


  - 警告 [YF]: 下载 TLT 数据失败 (尝试 2/3). 将在 1.00 秒后重试. 错误: yfinance returned an empty DataFrame for TLT.


d:\Project\Command\Python\Neural\Wolf_of_Wall_Street\data_process\get_data.py:308: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start_date, end=end_date_inclusive, progress=False)

1 Failed download:
['TLT']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


  - 警告 [YF]: 下载 TLT 数据失败 (尝试 3/3). 将在 2.00 秒后重试. 错误: yfinance returned an empty DataFrame for TLT.
  - 错误 [YF]: 下载 TLT 数据在 3 次尝试后仍失败。
--- 正在获取市场情绪数据 (恐慌指数) ---
  - 错误 [akshare]: 获取市场情绪数据失败: module 'akshare' has no attribute 'idx_ivix'
--- 所有全局市场数据准备完毕 ---


--- 正在为 TCL科技 (000100.SZ) 生成【训练用】历史特征 ---
  - 数据窗口: 2010-11-01 to 2025-09-30
  - 正在从本地缓存加载 TCL科技 (sz.000100) 的原始日线数据...
INFO: 开始特征计算流水线...
  - 将按以下阶段顺序执行计算器: ['base', 'contextual', 'composite']
  - [Calculating Features] INFO: No technical indicators specified in config. Skipping Technical Indicators.
  - [Calculating Features] INFO: No candlestick patterns specified in config. Skipping Candlestick Patterns.
  - [Calculating Features] Running: Calendar Features...
  - [Calculating Features] Running: Statistical Features...
  - [Calculating Features] Running: Price Structure Features...
  - [Calculating Features] Running: Volume Features...
  - [Calculating Features] Running: Trend Regime Features (Vectorized)...
  - [Calculating Fea

# **阶段二：模型训练与评估**

### 2.1 数据预加载与全局预处理 (L3 缓存)

In [3]:
if config and modules:
    # force_reprocess=True/False 可以控制是否重建缓存
    run_preprocess_l3_cache(config, modules, force_reprocess=False)

=== 工作流阶段 2.1：为模型预处理数据 (L3 缓存) ===
INFO: 开始执行预处理流程 (分块保存模式)...



正在预处理股票:   0%|          | 0/7 [00:00<?, ?it/s]


错误: 未找到 TCL科技 (000100.SZ) 的 L2 特征数据。跳过预处理。

错误: 未找到 兴业矿业 (000426.SZ) 的 L2 特征数据。跳过预处理。

错误: 未找到 孚日股份 (002083.SZ) 的 L2 特征数据。跳过预处理。

错误: 未找到 宜华健康 (000150.SZ) 的 L2 特征数据。跳过预处理。

错误: 未找到 新宁物流 (300013.SZ) 的 L2 特征数据。跳过预处理。

错误: 未找到 佳云科技 (300242.SZ) 的 L2 特征数据。跳过预处理。

错误: 未找到 精功科技 (002006.SZ) 的 L2 特征数据。跳过预处理。
--- INFO: L3 缓存文件已在磁盘上准备就绪。 ---
--- 阶段 2.1 成功完成。 ---


### 2.2 超参数优化

In [4]:
RUN_HPO = False
if RUN_HPO and config and modules:
    run_hpo_train(config, modules)

### 2.3 模型训练

In [5]:
if config and modules:
    all_ic_history = run_all_models_train(
        config, 
        modules,  
        force_retrain_base=False, 
        force_retrain_fuser=True
    )

=== 工作流阶段 2.3：训练所有模型 ===


处理股票模型:   0%|          | 0/7 [00:00<?, ?it/s]


ERROR: 未找到 000100.SZ 的 L3 缓存文件，跳过该股票的训练。

ERROR: 未找到 000426.SZ 的 L3 缓存文件，跳过该股票的训练。

ERROR: 未找到 002083.SZ 的 L3 缓存文件，跳过该股票的训练。

ERROR: 未找到 000150.SZ 的 L3 缓存文件，跳过该股票的训练。

ERROR: 未找到 300013.SZ 的 L3 缓存文件，跳过该股票的训练。

ERROR: 未找到 300242.SZ 的 L3 缓存文件，跳过该股票的训练。

ERROR: 未找到 002006.SZ 的 L3 缓存文件，跳过该股票的训练。

--- 阶段 2.3 成功完成。 ---


### 2.4 结果聚合、评估与可视化

In [6]:
if config and modules and 'all_ic_history' in locals() and all_ic_history:
    # 调用核心引擎中的评估和可视化函数
    evaluation_summary, backtest_summary, final_eval_df = run_performance_evaluation(config, modules, all_ic_history)
    
    # 只有在有结果时才进行可视化
    if evaluation_summary is not None or backtest_summary is not None:
        run_results_visualization(config, modules, evaluation_summary, backtest_summary, final_eval_df)
    else: print('无可视化结果')